In [ ]:
# import uproot3 as uproot
import uproot
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# plt.style.use("belle2")

import seaborn as sns

import sys

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
plt.rcParams.update({
    "axes.labelsize": 16,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 18
})

In [ ]:
# === Constants ===
m_D0 = 1.864  # GeV/c^2
m_e = 0.000511
m_pi = 0.13957

# === Fixed D⁰ lab-frame momentum ===
p_D0_lab = 2.0  # GeV/c
E_D0_lab = np.sqrt(p_D0_lab**2 + m_D0**2)
beta = p_D0_lab / E_D0_lab
gamma = E_D0_lab / m_D0

# === Number of events and plot font settings ===
n_events = 100_000
title_fontsize = 16  # Customize title font size here

# === Electron direction: flat in cos(theta), phi in D⁰ rest frame ===
cos_theta = np.random.uniform(-1, 1, n_events)
theta = np.arccos(cos_theta)
phi = np.random.uniform(0, 2 * np.pi, n_events)

# === Electron momentum in D⁰ rest frame ===
p_e_rf = np.random.uniform(0.0, 2.0, n_events)

# === Electron 3-momentum in D⁰ rest frame ===
px_rf = p_e_rf * np.sin(theta) * np.cos(phi)
py_rf = p_e_rf * np.sin(theta) * np.sin(phi)
pz_rf = p_e_rf * cos_theta

# === Electron energies (two hypotheses) ===
E_e_rf = np.sqrt(p_e_rf**2 + m_e**2)
E_pi_rf = np.sqrt(p_e_rf**2 + m_pi**2)

# === Boost to lab frame (z-axis only) ===
def boost_z(E, pz):
    E_lab = gamma * (E + beta * pz)
    pz_lab = gamma * (pz + beta * E)
    return E_lab, pz_lab

E_e_lab, pz_e_lab = boost_z(E_e_rf, pz_rf)
E_pi_lab, pz_pi_lab = boost_z(E_pi_rf, pz_rf)
px_lab = px_rf
py_lab = py_rf

# === Build 4-vectors in lab ===
p_e_lab = np.stack([E_e_lab, px_lab, py_lab, pz_e_lab], axis=1)
p_pi_lab = np.stack([E_pi_lab, px_lab, py_lab, pz_pi_lab], axis=1)
p_D0_lab_vec = np.array([E_D0_lab, 0, 0, p_D0_lab])

# === Total system: D⁰ + e+ ===
p_sum_e = p_e_lab + p_D0_lab_vec
p_sum_pi = p_pi_lab + p_D0_lab_vec

m_Ds_e = np.sqrt(p_sum_e[:, 0]**2 - p_sum_e[:, 1]**2 - p_sum_e[:, 2]**2 - p_sum_e[:, 3]**2)
m_Ds_pi = np.sqrt(p_sum_pi[:, 0]**2 - p_sum_pi[:, 1]**2 - p_sum_pi[:, 2]**2 - p_sum_pi[:, 3]**2)

delta_m_e = m_Ds_e - m_D0
delta_m_pi = m_Ds_pi - m_D0
delta_shift = delta_m_pi - delta_m_e

# === Other observables ===
p_e_mag_lab = np.sqrt(px_lab**2 + py_lab**2 + pz_e_lab**2)
cos_opening = pz_e_lab / p_e_mag_lab
theta_opening = np.arccos(np.clip(cos_opening, -1, 1))
p_D0_magnitude = np.full(n_events, p_D0_lab)

# === Plot 1: Delta m ===
plt.figure(figsize=(8, 6))
plt.hist(delta_m_e, bins=50, histtype='step', color='#2E2E2E', linewidth=2.5, label='Electron mass')
plt.hist(delta_m_pi, bins=50, histtype='step', color='#4C6EB1', linewidth=2.5, label='Pion mass hypothesis')
bin_width_dm = (np.max(delta_m_e) - np.min(delta_m_e)) / 50
plt.xlabel(r'$\Delta m(D_s^+ - D^0)$ [GeV/$c^2$]')
plt.ylabel(f'Entries / ( {bin_width_dm * 1000:.2f} MeV/$c^2$ )')
plt.title(r'$p_e^{\mathrm{D^0\!-\!rest}} \in [1.0,\, 2.0]~\mathrm{GeV}/c$', loc='left', fontsize=title_fontsize)
plt.title(r'N = 100,000', loc='right', fontsize=title_fontsize)
plt.legend()
plt.show()

# === Plot 2: Electron lab momentum ===
plt.figure(figsize=(8, 6))
plt.hist(p_e_mag_lab, bins=100, color='#4C6EB1', alpha=0.8)
bin_width_p_lab = (np.max(p_e_mag_lab) - np.min(p_e_mag_lab)) / 100
plt.xlabel(r'$|\vec{p}_e|$ in lab [GeV/$c$]')
plt.ylabel(f'Entries / ( {bin_width_p_lab * 1000:.2f} MeV/$c$ )')
plt.title('Electron Momentum in Lab Frame', fontsize=title_fontsize)
plt.show()

# === Plot 3: Opening angle in lab frame ===
plt.figure(figsize=(8, 6))
plt.hist(theta_opening, bins=100, color='#007C91', alpha=0.8)
bin_width_theta = (np.max(theta_opening) - np.min(theta_opening)) / 100
plt.xlabel(r'Opening angle between $D^0$ and $e^+$ [rad]')
plt.ylabel(f'Entries / ( {bin_width_theta:.3f} rad )')
plt.title('Opening Angle in Lab Frame', fontsize=title_fontsize)
plt.show()

# === Plot 4: cos(theta) in D⁰ rest frame ===
plt.figure(figsize=(8, 6))
plt.hist(cos_theta, bins=100, color='#007C91', alpha=0.9)
bin_width_costheta = (np.max(cos_theta) - np.min(cos_theta)) / 100
plt.xlabel(r'$\cos\theta$ in $D^0$ rest frame')
plt.ylabel(f'Entries / ( {bin_width_costheta:.3f} )')
plt.title(r'Isotropic $\cos\theta$ Distribution (Rest Frame)', fontsize=title_fontsize)
plt.show()

# === Plot 5: D⁰ momentum in lab frame (constant) ===
plt.figure(figsize=(8, 6))
plt.hist(p_D0_magnitude, bins=1, color='#007C91', alpha=1.0)
plt.xlabel(r'$|\vec{p}_{D^0}|$ in lab [GeV/$c$]')
plt.ylabel('Entries')
plt.title(r'Fixed $D^0$ Momentum in Lab Frame', fontsize=title_fontsize)
plt.show()

# === Plot 6: Electron momentum in D⁰ rest frame ===
plt.figure(figsize=(8, 6))
plt.hist(p_e_rf, bins=50, color='#4C6EB1', alpha=1.0)
bin_width_p_rest = (np.max(p_e_rf) - np.min(p_e_rf)) / 50
plt.xlabel(r'$|\vec{p}_e|$ in $D^0$ rest frame [GeV/$c$]')
plt.ylabel(f'Entries / ( {bin_width_p_rest * 1000:.2f} MeV/$c$ )')
plt.title('Electron Momentum in $D^0$ Rest Frame', fontsize=title_fontsize)
plt.show()

# === Plot 7: Shift vs Electron Momentum ===
fig, ax = plt.subplots(figsize=(8, 6))
h = ax.hist2d(p_e_rf, delta_shift, bins=[50, 50], cmap='plasma')
fig.colorbar(h[3], ax=ax, label='Entries')
ax.set_xlabel(r'$p_e^{\mathrm{D^0\!-\!rest}}$ [GeV/$c$]')
ax.set_ylabel(r'$\Delta m_\pi - \Delta m_e$ [GeV/$c^2$]')
ax.set_title(r'Shift in $\Delta m$: Pion mass $-$ Electron mass vs $p_e^{\mathrm{D^0\!-\!rest}}$')
plt.show()